<a href="https://colab.research.google.com/github/tarunsoni2112/OIBSIP/blob/main/Chat_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#create the server:
import socket
from threading import Thread

# Server configuration
SERVER_HOST = "0.0.0.0"
SERVER_PORT = 5002
clients = []
nicknames = []

# Create a TCP socket and bind it to the server address
def start():
    s = socket.socket()
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind((SERVER_HOST, SERVER_PORT))
    print(f"[*] Listening as {SERVER_HOST}:{SERVER_PORT}")
    s.listen(5)
    while True:
        c, addr = s.accept()
        print(f"[+] {addr} connected.")
        clients.append(c)
        c.send("NICK".encode())
        nickname = c.recv(1024).decode()
        nicknames.append(nickname)
        c.send(f"Welcome, {nickname}!".encode())
        print(f"The nickname of the client is {nickname}!")
        broadcast(f"{nickname} joined the chat!".encode(), c)
        thread = Thread(target=handle, args=(c,))
        thread.start()

# Handle messages from clients
def handle(client):
    while True:
        try:
            message = client.recv(1024).decode()
        except:
            index = clients.index(client)
            clients.remove(client)
            client.close()
            nickname = nicknames[index]
            nicknames.remove(nickname)
            broadcast(f"{nickname} left the chat!".encode(), client)
            break

        broadcast(message, client)

# Broadcast messages to all clients
def broadcast(message, sender):
    for client in clients:
        if client != sender:
            client.send(message)

if __name__ == "__main__":
    start()

In [ ]:
# create the client:
import socket
import threading

# Client configuration
SERVER_HOST = "127.0.0.1"
SERVER_PORT = 5002

# Create a TCP socket and connect to the server
def start():
    s = socket.socket()
    s.connect((SERVER_HOST, SERVER_PORT))
    print(f"[+] Connected to {SERVER_HOST}:{SERVER_PORT}")

    # Receive and handle messages from the server
    thread = threading.Thread(target=receive)
    thread.start()

    # Send messages to the server
    while True:
        message = f"{nickname}: {input('')}"
        s.send(message.encode())

# Receive messages from the server
def receive():
    while True:
        try:
            message = s.recv(1024).decode()
            if message == "NICK":
                s.send(nickname.encode())
            else:
                print(message)
        except:
            print("An error occurred!")
            s.close()
            break

# Set a name for the client
nickname = input("Enter your nickname: ")

# Start the client
start()